# Prompt Serialization

It is often preferrable to store prompts not as python code by as files. This notebook covers how to do that in LangChain, walking through all the different types of prompts and the different serialization options.

At a high level, the following design principles are applied to serialization:
1. Both JSON and YAML are supported. We want to support serialization methods are human readable on disk, and YAML and JSON are two of the most popular methods for that. Note that this rule applies to prompts. For other assets, like Examples, different serialization methods may be supported.
2. We support specifying everything in one file, or storing different components (templates, examples, etc) in different files and referencing them. For some cases, storing everything in file makes the most sense, but for others it is preferrable to split up some of the assets (long templates, large examples, reusable components). LangChain supports both.

In [8]:
# All prompts are loading through the `load_prompt` function.
from langchain.prompts.loading import load_prompt

## PromptTemplate

This section covers examples for loading a PromptTemplate.

### Loading from YAML
This shows an example of loading a PromptTemplate from YAML.

In [27]:
%%writefile simple_prompt.yaml
input_variables:
    ["adjective", "content"]
template: 
    Tell me a {adjective} joke about {content}.

Overwriting simple_prompt.yaml


In [28]:
prompt = load_prompt("simple_prompt.yaml")
print(prompt.format(adjective="funny", content="chickens"))

Tell me a funny joke about chickens.


### Loading from JSON
This shows an example of loading a PromptTemplate from JSON.

In [29]:
%%writefile simple_prompt.json
{
    "input_variables": ["adjective", "content"],
    "template": "Tell me a {adjective} joke about {content}."
}

Overwriting simple_prompt.json


### Loading Template from a File
This shows an example of storing the template in a separate file and then referencing it in the config. Notice that the key changes from `template` to `template_path`.

In [34]:
with open("simple_template.txt", 'w') as f:
    f.write("Tell me a {adjective} joke about {content}.")

In [35]:
%%writefile simple_prompt_with_template_file.json
{
    "input_variables": ["adjective", "content"],
    "template_path": "simple_template.txt"
}

Overwriting simple_prompt_with_template_file.json


In [36]:
prompt = load_prompt("simple_prompt_with_template_file.json")
print(prompt.format(adjective="funny", content="chickens"))

Tell me a funny joke about chickens.


## FewShotPromptTemplate

This section covers examples for loading few shot prompt templates.

### Examples
This shows an example of what examples stored as json might look like.

In [37]:
%%writefile examples.json
[
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"}
]

Writing examples.json


### Loading from YAML
This shows an example of loading a few shot example from YAML.

In [5]:
%%writefile few_shot_prompt.yaml
_type: few_shot
input_variables:
    ["adjective"]
prefix: 
    Write antonyms for the following words.
example_prompt:
    input_variables:
        ["input", "output"]
    template:
        "Input: {input}\nOutput: {output}"
examples:
    examples.json
suffix:
    "Input: {adjective}\nOutput:"

Overwriting few_shot_prompt.yaml


In [6]:
prompt = load_prompt("few_shot_prompt.yaml")
print(prompt.format(adjective="funny"))

Write antonyms for the following words.

Input: happy
Output: sad

Input: tall
Output: short

Input: funny
Output:


### Loading from JSON
This shows an example of loading a few shot example from JSON.

In [40]:
%%writefile few_shot_prompt.json
{
    "_type": "few_shot",
    "input_variables": ["adjective"],
    "prefix": "Write antonyms for the following words.",
    "example_prompt": {
        "input_variables": ["input", "output"],
        "template": "Input: {input}\nOutput: {output}"
    },
    "examples": "examples.json",
    "suffix": "Input: {adjective}\nOutput:"
}   

Writing few_shot_prompt.json


In [2]:
prompt = load_prompt("few_shot_prompt.json")
print(prompt.format(adjective="funny"))

Write antonyms for the following words.

Input: happy
Output: sad

Input: tall
Output: short

Input: funny
Output:


### Examples in the Config
This shows an example of referencing the examples directly in the config.

In [3]:
%%writefile few_shot_prompt_examples_in.json
{
    "_type": "few_shot",
    "input_variables": ["adjective"],
    "prefix": "Write antonyms for the following words.",
    "example_prompt": {
        "input_variables": ["input", "output"],
        "template": "Input: {input}\nOutput: {output}"
    },
    "examples": [
        {"input": "happy", "output": "sad"},
        {"input": "tall", "output": "short"}
    ],
    "suffix": "Input: {adjective}\nOutput:"
}   

Writing few_shot_prompt_examples_in.json


In [4]:
prompt = load_prompt("few_shot_prompt_examples_in.json")
print(prompt.format(adjective="funny"))

Write antonyms for the following words.

Input: happy
Output: sad

Input: tall
Output: short

Input: funny
Output:


### Example Prompt from a File
This shows an example of loading the PromptTemplate that is used to format the examples from a separate file. Note that the key changes from `example_prompt` to `example_prompt_path`.

In [5]:
%%writefile example_prompt.json
{
    "input_variables": ["input", "output"],
    "template": "Input: {input}\nOutput: {output}" 
}

Writing example_prompt.json


In [3]:
%%writefile few_shot_prompt_example_prompt.json
{
    "_type": "few_shot",
    "input_variables": ["adjective"],
    "prefix": "Write antonyms for the following words.",
    "example_prompt_path": "example_prompt.json",
    "examples": "examples.json",
    "suffix": "Input: {adjective}\nOutput:"
}   

Overwriting few_shot_prompt_example_prompt.json


In [2]:
prompt = load_prompt("few_shot_prompt_example_prompt.json")
print(prompt.format(adjective="funny"))

Write antonyms for the following words.

Input: happy
Output: sad

Input: tall
Output: short

Input: funny
Output:
